In [1]:
%matplotlib notebook
%reset -f


import pandas as pd
import numpy as np
from scipy.stats import norm, probplot
import statsmodels.api as sm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression

In [2]:
#for c in range(1,len(db.columns)):
    #db[db.columns[c]] = db[db.columns[c]].astype('category')



def regress(X,Y):
    #X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X)
    results = model.fit()
    return results


def dummify(data, column_name):
    """
        Converte a coluna column_name em dummies / one-hot e as adiciona ao dataframe
        retorna uma copia do  df original *sem* a coluna que foi dummified
    """
    df = data.copy()
    # Nota: se você for um econometrista, pode se interessar por usar a opção dropFirst = True na get_dummies
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

def dummify2(data, column_name):
    return pd.get_dummies(data[column_name], prefix=column_name)
    


def variance_threshold_selector(data, threshold):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [3]:
db = pd.read_excel('new_vdem.xlsx')
db.set_index("Country")

,VDEM,1,3,5,7,9,11,13,16,17,...,54,55,56,57,58,59,60,61,62,63
Country,,,,,,,,,,,,,,,,,,,,,
Angola,0.386,#Yes,#No,#No,#No,#No,#Yes,#Legal entities\n#Other,#No,#Not applicable,...,#Yes,#Yes,#Yes,"#Yes, EMB",#EMB,#No data,#No data,#No data,#Court,#Fines\n#Other\n#Suspension of public funding
Argentina,0.812,#Yes,#Yes,#Yes,#Yes,#Yes,#Yes,#Organisations connected to gambling\n#Organis...,"#Yes, for both natural and legal persons","#For juridical persons, not above 1% of the to...",...,#Yes,#Yes,#Yes,#Yes,#Federal Justice with electoral jurisdiction,#Impose fines,#Not applicable,#No,#Body of Auditors,#Fines #Loss of public funding #Loss of politi...
Australia,0.838,#Yes,#No,#No,"#No, but specific limit",#No,#No,#No,#No,#Not applicable,...,#Sometimes,#Yes,#Yes,"#Yes, EMB",#EMB,#Refer for investigation\n#Carry out investiga...,#Not applicable,#No,#Other,#Fines\n#Prison
Belgium,0.882,#No,#Yes,#Yes,"#No, but specific limit",#Yes,#Yes,#Legal entities,"#Yes, for natural persons",#A party may receive maximum EUR 500 from an i...,...,#Yes,#Yes,#Yes,"#Yes, other\n#Yes, institution for this purpose",#Ministry\n#Parliamentary unit,#Impose sanctions,#Not applicable,#No data,#Court,#Fines\n#Loss of public funding
Brazil,0.674,#Yes,#Yes,#Yes,#Yes,#Yes,#Yes,#Religious organisations\n#Charities\n#Other,"#Yes, for natural persons",#10% of the profits earned by the donor in the...,...,#Yes,#Yes,#Yes,"#Yes, EMB",#EMB,#Carry out investigation\n#Request additional ...,#Refer for investigation\n#Carry out investiga...,#Yes,#Court\n#Other,#Fines\n#Loss of public funding\n#Deregistrati...
Canada,0.866,#Yes,#Yes,#Yes,#Yes,#Yes,#Yes,#Other,"#Yes, for natural persons",#This amount increases by $25 on January 1 in ...,...,#Yes,#Yes,#Yes,"#Yes, EMB",#EMB,#Carry out investigation\n#Request additional ...,#Not applicable,#No,#None,#Fines\n#Prison\n#Forfeiture\n#Deregistration ...
Chile,0.773,#Yes,#Yes,#Yes,"#No, but specific limit",#Yes,#Yes,#Legal entities\n#Religious organisations\n#Or...,"#Yes, for natural persons","#300 indexed units/ per year ( non members), 5...",...,#Sometimes,#Yes,#Yes,"#Yes, EMB",#EMB,#Refer for investigation\n#Carry out investiga...,#Carry out investigation\n#Request additional ...,#Yes,#Auditing agency,#Fines\n#Prison\n#Deregistration of party\n#Su...
Colombia,0.667,#Yes,#No,#No,#Yes,#Yes,#Yes,#Organisations connected to gambling\n#Other,#No,#Not applicable,...,#Yes,#Yes,#Yes,"#Yes, EMB",#EMB,#Refer for investigation\n#Carry out investiga...,#Refer for investigation\n#Carry out investiga...,#Yes,#Court,#Fines\n#Prison\n#Forfeiture\n#Loss of public ...
Czech Republic,0.796,#Yes,#No,#No,#Yes,#No,#Yes,#Other,"#Yes, for both natural and legal persons","#CZK 3000000 in one calendar year, both for na...",...,#Yes,#Yes,#Yes,"#Yes, institution for this purpose",#Special institution,#Request additional information from potential...,#No data,#No data,#Ministry,#Fines\n#Loss of public funding


In [4]:
db.columns

Index(['Country',    'VDEM',         1,         3,         5,         7,
               9,        11,        13,        16,        17,        18,
              19,        23,        25,        26,        39,         2,
               4,         6,         8,        10,        12,        20,
              21,        22,        24,        27,    '13.1',        30,
              31,        32,        33,        37,        38,        14,
              15,        34,        35,        36,        40,        41,
              42,        43,        44,        45,        46,        47,
              48,        28,        29,        49,        50,        51,
              52,        53,        54,        55,        56,        57,
              58,        59,        60,        61,        62,        63],
      dtype='object')

In [5]:
df = db[['VDEM',1,         3,         5,         7,
               9,        11,        13,        16,        18,        23,
              25,        26,        39,         2,         4,         6,
               8,        10,        12,        20,        22,        24,
              27,    '13.1',        30,        31,        32,        33,
              37,        38,        14,        15,        34,        35,
              36,        40,        41,        43,        45,        46,
              47,        48,        28,        29,        49,        50,
              51,        52,        53,        54,        55,        56,
              57,        58,        59,        60,        61,        63]]

In [6]:
Y = df['VDEM']
X = df[[1,         3,         5,         7,
               9,        11,        13,        16,        18,        23,
              25,        26,        39,         2,         4,         6,
               8,        10,        12,        20,        22,        24,
              27,    '13.1',        30,        31,        32,        33,
              37,        38,        14,        15,        34,        35,
              36,        40,        41,        43,        45,        46,
              47,        48,        28,        29,        49,        50,
              51,        52,        53,        54,        55,        56,
              57,        58,        59,        60,        61,        63]]

In [7]:
dici = {}
rsquared = []
questions = []
for c in X.columns:
    dvar = dummify2(X, c)
    clf = regress(dvar,Y)
    rsquared.append(clf.rsquared)
    questions.append(c)

dici['Rsquared'] = rsquared
dici['Questions'] = questions

frame = pd.DataFrame.from_dict(dici)
frame = frame.sort_values(by='Rsquared', ascending=False)
frame = frame.head(10)
X = df[list(frame["Questions"])]
for c in X.columns:
    X = dummify(X, c)
X

,8_#No,"8_#No, but specific limit",8_#Yes,2_#No,2_#No data,2_#Yes,24_#No,24_#No data,24_#Yes,24_#Yes. Donation limit applies.,...,51_#No,51_#Yes,12_#No,12_#No data,12_#Yes,6_#No,6_#Yes,43_#No,43_#Only presidential candidates who accept public funds must agree to abide by the spending limit.,43_#Yes
0,1,0,0,0,0,1,1,0,0,0,...,0,1,0,0,1,1,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,1,1,0,0,0,...,0,1,1,0,0,1,0,1,0,0
3,0,1,0,1,0,0,0,1,0,0,...,0,1,0,0,1,0,1,0,0,1
4,0,0,1,0,0,1,1,0,0,0,...,0,1,0,0,1,0,1,0,0,1
5,0,0,1,0,0,1,0,0,1,0,...,0,1,0,0,1,0,1,0,0,1
6,0,1,0,0,0,1,0,0,1,0,...,0,1,0,0,1,0,1,0,0,1
7,0,0,1,0,0,1,0,0,1,0,...,0,1,0,0,1,1,0,0,0,1
8,0,0,1,0,0,1,1,0,0,0,...,0,1,0,0,1,1,0,0,0,1
9,1,0,0,1,0,0,1,0,0,0,...,0,1,1,0,0,1,0,1,0,0


In [8]:


results = regress(X,Y)
# Regression with selected features
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   VDEM   R-squared (uncentered):                   0.954
Model:                            OLS   Adj. R-squared (uncentered):              0.894
Method:                 Least Squares   F-statistic:                              15.87
Date:                Sat, 18 Jul 2020   Prob (F-statistic):                    4.28e-07
Time:                        16:42:18   Log-Likelihood:                          14.480
No. Observations:                  37   AIC:                                      13.04
Df Residuals:                      16   BIC:                                      46.87
Df Model:                          21                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================================================================================================
                                                                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
8_#No                                                                                                  -0.1098      0.112     -0.983      0.340      -0.346       0.127
8_#No, but specific limit                                                                               0.2795      0.117      2.399      0.029       0.033       0.526
8_#Yes                                                                                                  0.1288      0.085      1.521      0.148      -0.051       0.308
2_#No                                                                                                   0.1071      0.160      0.670      0.513      -0.232       0.446
2_#No data                                                                                              0.2123      0.294      0.723      0.480      -0.411       0.835
2_#Yes                                                                                                 -0.0209      0.177     -0.119      0.907      -0.395       0.353
24_#No                                                                                                  0.1562      0.153      1.023      0.321      -0.167       0.480
24_#No data                                                                                            -0.1028      0.261     -0.393      0.699      -0.657       0.451
24_#Yes                                                                                                 0.0139      0.128      0.109      0.914      -0.256       0.284
24_#Yes. Donation limit applies.                                                                        0.1193      0.248      0.481      0.637      -0.407       0.645
24_#Yes. Regular limits apply.                                                                          0.1118      0.121      0.925      0.369      -0.144       0.368
20_#No                                                                                                 -0.0418      0.213     -0.196      0.847      -0.494       0.410
20_#Yes, for both natural and legal persons                                                             0.1161      0.128      0.909      0.377      -0.155       0.387
20_#Yes, for legal persons                                                                              0.1590      0.225      0.705      0.491      -0.319       0.637
20_#Yes, for natural persons                                                                            0.0652      0.138      0